In [1]:
import psycopg2
import configparser
import pandas as pd
import matplotlib.pyplot as plt
from sql_queries import create_table_queries, drop_table_queries, copy_table_queries, insert_table_queries

## postgresql connection

In [2]:
def create_database():

    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=antoniam")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("""
    SELECT pg_terminate_backend(pg_stat_activity.pid)
    FROM pg_stat_activity
    WHERE pg_stat_activity.datname = 'movies_db';""")
    cur.execute("DROP DATABASE IF EXISTS movies_db")
    cur.execute("CREATE DATABASE movies_db WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=movies_db")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

## ETL

In [3]:
def drop_tables(cur):
    """
        Drops all the tables

        Parameters:
        - cur: cursor object that allows Python to execute PostgreSQL commands in a database session
        - conn: connection to the cluster
    """
    for query in drop_table_queries:
        cur.execute(query)


def create_tables(cur):
    """
        Creates all the tables

        Parameters:
        - cur: cursor object that allows Python to execute PostgreSQL commands in a database session
        - conn: connection to the cluster
    """
    for query in create_table_queries:
        cur.execute(query)

def load_staging_tables(cur):
    """
        Extracts data into staging tables

        Parameters:
        - cur: cursor object that allows Python to execute PostgreSQL commands in a database session
        - conn: connection to the cluster
    """
    for query in copy_table_queries:
        print(query)
        cur.execute(query)


def insert_tables(cur):
    """
        Inserts (transformed) data into final tables from staging tables

        Parameters:
        - cur: cursor object that allows Python to execute PostgreSQL commands in a database session
        - conn: connection to the cluster
    """
    for query in insert_table_queries:
        cur.execute(query)

## Quality Checks

In [4]:
def staging_tables_quality_checks():
    staging_tables = ['staging_imdb_titles', 'staging_imdb_ratings', 'staging_imdb_principals',
                     'staging_imdb_crew', 'staging_imdb_names', 'staging_rotten_tomatoes_titles', 'staging_netflix_titles']
    for table in staging_tables:
        cur.execute(f'select count(*) from {table} LIMIT 10')
        if cur.fetchone()[0] < 1:
            raise Exception (f'Records did not get copied correctly for table {table}')
    print('Staging Quality Checks completed successfully')
        
def final_table_quality_checks():
    def netflix_titles_match():
        cur.execute('SELECT DISTINCT show_id from staging_netflix_titles')
        netflix_titles_staging = cur.fetchall()

        cur.execute('SELECT DISTINCT netflix_id from titles')
        netflix_titles_final = cur.fetchall()
        if sorted(netflix_titles_staging) != sorted(netflix_titles_final):
            raise Exception('Missing or additional netflix titles')
        print('Test 1: Netflix titles match')
        
    def correct_multiple_roles():
        cur.execute("""SELECT role_name from titles 
                        JOIN roles
                        ON titles.imdb_id = roles.title_id
                        JOIN persons
                        ON persons.id = roles.person_id
                        where netflix_id='s1156' AND full_name='Angelina Jolie'
            """)
        res = cur.fetchall()
        if sorted([r[0] for r in res]) != ['actress', 'director', 'writer']:
            raise Exception('Role/Person tables incorrectly created')
        print('Test2: Correct multiple roles')
    netflix_titles_match()
    correct_multiple_roles()

In [5]:
cur, conn = create_database()

In [ ]:
print('Cleaning up tables')
drop_tables(cur)
print('Creating tables')
create_tables(cur)
print('Loading staging tables')
load_staging_tables(cur)
print('Staging tables Quality Checks')
staging_tables_quality_checks()
print("Inserting tables")
insert_tables(cur)

Cleaning up tables
Creating tables
Loading staging tables
COPY staging_imdb_titles FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.basics.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_ratings FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.ratings.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_principals FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.principals.tsv' delimiter '	' CSV HEADER


In [ ]:
print('Running final quality checks')
final_table_quality_checks()

## EXAMPLE QUERIES

### 1- Get horror movies released after 2016 with rotten tomatoes critics score higher than 70, ordered by imdb score

In [ ]:
cur.execute("""SELECT name, imdb_avg_score, rt_critics_score, genre_name
            from titles 
            join genres on titles.imdb_id = genres.title_id
            where rt_critics_score > 70 and release_year > 2016 
            and imdb_avg_score is not NULL
            and genre_name='Horror'
            order by imdb_avg_score DESC
            limit 20""")
cur.fetchall()

### 2- Find rotten tomatoes average critics score for all the netflix movies per between year 2000 and 2019

In [ ]:
cur.execute("""SELECT release_year, avg(rt_critics_score)
            FROM titles
            where release_year between 2000 AND 2019
            GROUP BY release_year
            ORDER BY release_year""")
cur.fetchall()

### 3- Is there a correlation between:
- imdb and critics score?
- critics score and audience score?

In [ ]:
cur.execute("""SELECT imdb_avg_score, rt_critics_score, rt_audience_score
            FROM titles
            WHERE rt_critics_score is not Null 
            AND rt_audience_score is not Null
            AND imdb_avg_score is not Null""")
res = cur.fetchall()

In [ ]:
imdb_avg_score = [r[0] for r in res]
rt_critics_score = [r[1] for r in res]
rt_audience_score = [r[2] for r in res]

In [ ]:
plt.scatter(imdb_avg_score, rt_critics_score)
plt.xlabel('imdb score')
plt.ylabel('rt_critics score')
plt.show()

In [ ]:
plt.scatter(rt_critics_score, rt_audience_score)
plt.xlabel('rt_critics score')
plt.ylabel('rt_audience score')
plt.show()